In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("10-monkey-species"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'training', 'monkey_labels.txt', 'validation']


In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import sys
import tensorflow as tf
import time
from tensorflow import keras

print(tf.__version__)

2.0.0


In [3]:
train_dir = "10-monkey-species/training/training"
valid_dir = "10-monkey-species/validation/validation"
label_file = "10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

True
True
True
['n0', 'n7', 'n9', 'n8', 'n6', 'n1', 'n4', 'n3', 'n2', 'n5']
['n0', 'n7', 'n9', 'n8', 'n6', 'n1', 'n4', 'n3', 'n2', 'n5']


In [4]:
labels = pd.read_csv(label_file, header=0)
print(labels)

   Label     Latin Name              Common Name                     \
0  n0         alouatta_palliata\t    mantled_howler                   
1  n1        erythrocebus_patas\t    patas_monkey                     
2  n2        cacajao_calvus\t        bald_uakari                      
3  n3        macaca_fuscata\t        japanese_macaque                 
4  n4       cebuella_pygmea\t        pygmy_marmoset                   
5  n5       cebus_capucinus\t        white_headed_capuchin            
6  n6       mico_argentatus\t        silvery_marmoset                 
7  n7      saimiri_sciureus\t        common_squirrel_monkey           
8  n8       aotus_nigriceps\t        black_headed_night_monkey        
9  n9       trachypithecus_johnii    nilgiri_langur                   

    Train Images    Validation Images  
0             131                  26  
1             139                  28  
2             137                  27  
3             152                  30  
4             131 

In [5]:
# 定义图片大小
height = 128
width = 128
channels = 3
batch_size = 64
num_classes = 10

# 创建数据generator（针对图片的api）
# 里面的参数都是做数据增强的
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,    # 缩放数据，将每个值除以255
                                                             rotation_range = 40, # 旋转角度，随机旋转（-40， 40）度
                                                             width_shift_range = 0.2,   # 水平位移 
                                                             height_shift_range = 0.2,  # 垂直位移
                                                             shear_range = 0.2, # 剪切强度
                                                             zoom_range = 0.2, # 缩放强度
                                                             horizontal_flip = True, # 是否随机做水平翻转
                                                             fill_mode = 'nearest'  # 填充像素策略
                                                              )
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = True,
                                                    class_mode = 'categorical' # 使用one_hot编码
                                                    )
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_directory(valid_dir, 
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = True,
                                                    class_mode = 'categorical' # 使用one_hot编码
                                                    )

train_num = train_generator.samples  # 训练集数据数量
valid_num = train_generator.samples
print(train_num, valid_num)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 1098


In [6]:
# 从generator中取数据
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(64, 128, 128, 3) (64, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.

In [7]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=[width, height, channels]),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='selu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 128)       7

In [8]:
epochs = 10
history = model.fit_generator(train_generator, 
                              steps_per_epoch = train_num // batch_size, 
                              epochs=epochs, 
                              validation_data=valid_generator,
                              validation_steps=valid_num // batch_size
                             )

Epoch 1/10
17/17 [==============================] - 114s 7s/step - loss: 2.3906 - accuracy: 0.1422 - val_loss: 2.2545 - val_accuracy: 0.2288
Epoch 2/10
17/17 [==============================] - 125s 7s/step - loss: 2.1701 - accuracy: 0.2021 - val_loss: 1.9802 - val_accuracy: 0.2617
Epoch 3/10
17/17 [==============================] - 128s 8s/step - loss: 2.0173 - accuracy: 0.2718 - val_loss: 1.8112 - val_accuracy: 0.3337
Epoch 4/10
17/17 [==============================] - 134s 8s/step - loss: 1.9793 - accuracy: 0.2941 - val_loss: 1.9410 - val_accuracy: 0.3008
Epoch 5/10
17/17 [==============================] - 122s 7s/step - loss: 1.8752 - accuracy: 0.3153 - val_loss: 1.5709 - val_accuracy: 0.4121
Epoch 6/10
17/17 [==============================] - 126s 7s/step - loss: 1.7832 - accuracy: 0.3743 - val_loss: 1.5356 - val_accuracy: 0.4502
Epoch 7/10
17/17 [==============================] - 126s 7s/step - loss: 1.6722 - accuracy: 0.3786 - val_loss: 1.5823 - val_accuracy: 0.3814
Epoch 8/10
17

In [ ]:
print(history.history.keys())

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()

plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2.5)